# Model-data calibration for the DVM-DOS-TEM model using [MADS](https://github.com/madsjulia/Mads.jl) julia

This exmaple illustrates usage of the MADS (Model Analysis & Decision Support) package. Here, we use MADS package for the data-model calibration, i.e. minimize the difference between observations and the corresponding modeled outputs. In this exmaple we wrap the DVM-DOS-TEM model with MADS to calibrate model parameters by comparing model outputs with observations from the Murphy Dome (black spruce) site near Fairbanks, Alaska. This example is for STEP1 only and assumes that a user has a background on 6 calibration steps and has all the required packages installed. Note, that this example runs only from docker containter. 

## STEP1-MD1 
In STEP1, we calibrate parameter `cmax` for four plant functional types (PFT) by tweaking `cmax` values in the `parameters/cmt_calparbgc.txt` file. Our target values Gross Primary Productivity (GPP) values. The observed GPP values can be found in `calibration/calibration_targets.py`. 

In [ ]:
# STEP1 (MD1)
# parameters: cmax
# targets: GPP[1:4], Nitrogen unlimited mode

import Mads
import PyCall
@show pwd()

For **STEP1**, we setup the nitrogen unlimited mode. This mode can be enabled by setup the `calib_mode` eqaul to `'GPPAllIgnoringNitrogen'`. Below we embedded the python code inside the julia notebook. `PyCall` package provide the communication between `python` and `julia` languages. Similar simplified example that uses python from julia can be found on the MADS Github [page](https://github.com/madsjulia/Mads.jl/blob/master/notebooks/model_diagnostics_python/model_diagnostics_python.jl). Most of the utility functions required to run the calibration example live in `TEM.py`, which is located in `work/scripts` folder. The community number type (CMT) correspoding to black spruce forest is `cmtnum=1`. The code below uses two functions: `run_TEM` runs the model and grabs the outputs, and `get_targets` grabs the correspoding target values (in this case GPPs) from `calibration/calibration_targets.py` file. Typically vegetation carbon (above ground carbon) takes less time to equilibrate, that is why we use 100 year of pre-run and 200 of equilibrium run, which should suffy for the model to rich equilibrium state for vegetation. 

In [ ]:
PyCall.py"""

import sys,os
sys.path.append(os.path.join('/work','scripts'))
import TEM

def run_TEM(x):
    
    # update param files
    for j in range(len(dvmdostem.params)):
        dvmdostem.params[j]['val']=x[j]   
    
    dvmdostem.clean()             # clean results from previous run
    dvmdostem.setup(calib=True)   # setup for a new run
    dvmdostem.update_params()     # save updated paramters into the cmt_calparbgc.txt
    dvmdostem.run()               # run the model

    return dvmdostem.get_calibration_outputs()[:4]

def get_targets():
    return dvmdostem.get_calibration_outputs(calib=True)[:4]

# istantiate the model
dvmdostem=TEM.TEM_model()
# set the path the input data  
dvmdostem.site='/data/input-catalog/cru-ts40_ar5_rcp85_mri-cgcm3_MurphyDome_10x10'
# set the path to the ouput folder
dvmdostem.work_dir='/data/workflows/MD1'
# set the mode
dvmdostem.calib_mode='GPPAllIgnoringNitrogen'
# set the run setup
dvmdostem.opt_run_setup='--pr-yrs 100 --eq-yrs 200 --sp-yrs 0 --tr-yrs 0 --sc-yrs 0'
# define parameters which will participate in calibration 
dvmdostem.set_params(cmtnum=1, params=['cmax','cmax','cmax','cmax'], \
                               pftnums=[0,1,2,3])
"""

Define the initial guess (IG) vector. Typically comes from the `cmt_calparbgc.txt`. These IG values need to be updated during the calibration process. 

In [ ]:
initial_guess=[385.0, 115.0, 201.0, 95.0]
# the y_init line runs the run_TEM functions in the /data/workflows/MD1 folder
# y_init corresponds to the modeled GPP outputs
y_init=PyCall.py"run_TEM"(initial_guess)

Let's grab the target GPP values and get their length. 

In [ ]:
targets=PyCall.py"get_param_targets"()
obs_time=1:length(targets)

Ths function below serves as a broker between `MADS` julia and python's `run_TEM` function. 

In [ ]:
function TEM_pycall(parameters::AbstractVector)
        predictions = PyCall.py"run_TEM"(parameters)
        return predictions
end

Below we set the calibration problem (`createproblem`) to minimize the difference between GPP observed and GPP modeled. `paramdist` sets up the range for the calibrated parameters (see `paramkey`). 

In [ ]:
md = Mads.createproblem(
    initial_guess,               #IG vector to start the calibration
    targets,                     #obsevations
    TEM_pycall;                  #GPP modeled
    paramkey=["cmax0","cmax1","cmax2","cmax3"],
    paramdist=["Uniform(0.1, 500)","Uniform(0.1, 200)","Uniform(0.1, 300)","Uniform(0.1, 200)"],
    obstime=obs_time,            #length of the observation vector
    obsweight=[100,100,100,100], #weight for target values
    problemname="STEP1-MD1")     

Dislpay parameter values and observations (targets). 

In [ ]:
Mads.showparameters(md)
Mads.showobservations(md)

Calibrate functions will run the calibration algorithm. In this case, by default it is using [Levenberg-Marquardt Algorithm](https://en.wikipedia.org/wiki/Levenberg%E2%80%93Marquardt_algorithm) to minimize the difference between modeled and observed GPPs. `tolOF` sets the tolerance number for the minimization function. `OF` stands for th root mean square difference between observations and modeled outputs.  `tolOFcount` sets the number of times after which it stops if there is no improvements in the `OF` number. This step could take a while to finish.   

In [ ]:
calib_param, calib_information = Mads.calibrate(md, tolOF=0.01, tolOFcount=4)

Plot the match between observation and modeled outputs

In [ ]:
Mads.plotmatches(md, calib_param, 
    xtitle="# of observations", ytitle="GPP",filename="STEP1-MD1-matchplot.png")

In [ ]:
localsa = Mads.localsa(md; filename="model_diagnostics.png", par=collect(values(calib_param)))

In [ ]:
[Mads.getparamlabels(md) localsa["stddev"]]

In [ ]:
Mads.display("model_diagnostics-jacobian.png")

In [ ]:
Mads.display("model_diagnostics-eigenmatrix.png")

In [ ]:
Mads.display("model_diagnostics-eigenvalues.png")